In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
# faster rcnn model이 포함된 library
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm

import json
import pandas as pd
from collections import defaultdict

In [2]:
def make_dataframe(target):
    # target 에 존재하는 train.json 파일을 엽니다.
    with open(target, 'r') as f:
        json_datas = json.load(f) # python dict 처럼 접근하게끔 변환
    	#dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])

    category = {}
    file_info = {}
    make_frame = defaultdict(list)

    # 이미지 정보 중 파일 경로와 아이디만 추출해서 file_info 에 저장
    for item in json_datas['images']:
        file_info[item['id']] = {'id' : item['id'], 'file_name' : item['file_name']}
    # 카테고리 정보를 category 에 저장
    for item in json_datas['categories']:
        category[item['id']] = item['name']
    # annotations 에 속하는 아이템들을 images 에 속하는 아이템의 정보와 합치기 위함
    for annotation in json_datas['annotations']:
        save_dict = file_info[annotation['image_id']]
        # 각 이미지에 해당하는 bounding box 정보와 class 정보 area(넓이) 정보를 추가
        bbox = np.array(annotation['bbox'])
        bbox[2] = bbox[2] + bbox[0]
        bbox[3] = bbox[3] + bbox[1]
        save_dict.update({
            'class': annotation['category_id'], # 배경은 0, 나머지 +1 in faster_rcnn
            'x_min': bbox[0],
            'y_min': bbox[1],
            'x_max': bbox[2],
            'y_max': bbox[3],
            'area':annotation['area']
            })

        for k,v in save_dict.items():
            # dataframe 으로 만들기 위해서 'key' : [item1,item2...] 형태로 저장
            make_frame[k].append(v)

    # dictionary 가 잘 만들어 졌는지 길이를 측정해서 확인해보세요!
    print(len(json_datas['annotations']))
    # dictionary to DataFrame
    df = pd.DataFrame.from_dict(make_frame)
    df.to_csv('./detection_info.csv',index=False)
    print(df.head())

    return df

In [3]:
annotation = '../dataset/train.json' # annotation 경로
train_df = make_dataframe(annotation)

23144
   id       file_name  class  x_min  y_min  x_max  y_max       area
0   0  train/0000.jpg      0  197.6  193.7  745.4  663.4  257301.66
1   1  train/0001.jpg      3    0.0  407.4   57.6  588.0   10402.56
2   1  train/0001.jpg      7    0.0  455.6  144.6  637.2   26259.36
3   1  train/0001.jpg      4  722.3  313.4  996.6  565.3   69096.17
4   1  train/0001.jpg      5  353.2  671.0  586.9  774.4   24164.58


In [4]:
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict


In [5]:
def stratified_group_k_fold(X, y, groups, k, seed=None):

    #stratified_group_k_fold(train_x, train_y, groups, k=5)

    labels_num = np.max(y) + 1 # class num ()
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num)) # 그룹마다 클래스의 수 분포를 파악
    y_distr = Counter() # 모든 라벨의 개수를 세어서 dict 형태로 반환 

    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1 # 그룹마다 라벨의 위치에 클래스 개수를 늘려준다.
        y_distr[label] += 1 # 총 라벨의 개수 증가
    #print(y_distr)

    #print(y_counts_per_group["train/4882.jpg"])
    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num)) # fold마다 클래스의 수 분포를 파악
    groups_per_fold = defaultdict(set) # fold별 group 만들기

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts # fold 하나에 Image label이 더해집니다. (계산하기 위한 용도?, Numpy는 list끼리 더하기가 가능하다.
        std_per_label = []
        #print(y_counts_per_fold[fold])
        for label in range(labels_num): # 10개
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)]) # 모든 fold에 있는 라벨들에 대해 비율을 구하고 그 표준편차를 구한다.
            std_per_label.append(label_std) # 라벨 당 표준편차들을 구합니다.
            
        y_counts_per_fold[fold] -= y_counts # fold당 모든 이미지의 라벨을 다시 뺴줍니다.
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items()) # list화  [file_name, label_list]
    random.Random(seed).shuffle(groups_and_y_counts) # list random shuffle 섞을 필요가 있나? 밑에서 sort를 하는데..?

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])): # image당 라벨 개수들의 분포에 -표준편차를 기준으로 sort
    #for g, y_counts in groups_and_y_counts:
        best_fold = None
        min_eval = None
        for i in range(k): # k: fold 개수
            fold_eval = eval_y_counts_per_fold(y_counts, i) # group이 가지고 있는 라벨에서 / fold 별로 각 label의 표준 편차를 구하고 / 그룹에 있는 라벨들의 표준 편차의 평균을 구한다.
            if min_eval is None or fold_eval < min_eval: # 라벨들의 표준편차 그리고 그에 평균 값이 가장 적은 곳에 best_fold를 해준다.
                min_eval = fold_eval
                best_fold = i

        # 결국 라벨들의 분포를 골고루 넣어주기 위한 과정

        y_counts_per_fold[best_fold] += y_counts  # 폴드 당 라벨들의 합이 들어있따.
        groups_per_fold[best_fold].add(g) # 폴드당 이미지들을 다 넣어줌

    all_groups = set(groups) # 중복 그룹 제거
    for i in range(k): # fold = 0, 1, 2, 3, 4 일 때,train/test group 만들기
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices


In [6]:
train_x = train_df["id"]
train_y = train_df["class"]
groups = train_df["file_name"]

def get_distribution(y_vals):
        y_distr = Counter(y_vals)
        y_vals_sum = sum(y_distr.values())
        return [f'{y_distr[i] / y_vals_sum:.2%}' for i in range(np.max(y_vals) + 1)]


In [7]:



# target 에 존재하는 train.json 파일을 엽니다.

def make_json(fold_ind, dev_id_json, val_id_json):

    train_path = "/opt/ml/detection/dataset/train.json"
    train_file_name = f"../dataset/fold_test/fold{fold_ind}_train.json"
    valid_file_name = f"../dataset/fold_test/fold{fold_ind}_valid.json"
    
    # print(fold_ind)
    # print(dev_id_json[0])
    # print(val_id_json[0])
    with open(train_path, 'r') as f:
        json_datas = json.load(f) # python dict 처럼 접근하게끔 변환
    	#dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])
    # category = {}
    # file_info = {}

    info = json_datas["info"]
    licenses = json_datas["licenses"]
    categories = json_datas["licenses"]

    images_train = []
    images_valid = []

    annotations_train = []
    annotations_valid = []

    for item in json_datas["images"]:
        if item["id"] in dev_id_json:
            images_train.append(item)
        elif item["id"] in val_id_json:
            images_valid.append(item)
        else:
            print("no id in images")


    for item in json_datas["annotations"]:
        if item["image_id"] in dev_id_json:
            annotations_train.append(item)
        elif item["image_id"] in val_id_json:
            annotations_valid.append(item)
        else:
            print("no id in annotations")

    make_json_train = defaultdict(list)
    make_json_valid = defaultdict(list)
    make_json_train = {"info":info, "licenses" : licenses, "images": images_train, "categories":categories, "annotations":annotations_train}
    make_json_valid = {"info":info, "licenses" : licenses, "images": images_valid, "categories":categories, "annotations":annotations_valid}

    # 개수 파악
    if len(json_datas["images"]) != len(make_json_train["images"]) + len(make_json_valid["images"]):
        print("images length, diff")

    if len(json_datas["annotations"]) != len(make_json_train["annotations"]) + len(make_json_valid["annotations"]):
        print("annotations length, diff")

    # 이름 중복 파악 - > 모든 성분 파악
    # for item in make_json_train["images"] (dict):
    #     if item["file_name"] in make_json_valid["images"]:
    #         print("same file name exist")
    
    # for item in make_json_train["annotations"]:
    #     if item["file_name"] in make_json_valid["annotations"]:
    #         print("same file name exist")

    #assert len(set(make_json_train["annotations"]) & set(make_json_valid["annotations"])) == 0

    #print(make_json)
    
    
    with open(train_file_name, 'w') as output:
        json.dump(make_json_train, output, indent=2)

    with open(valid_file_name, 'w') as output:
        json.dump(make_json_valid, output, indent=2)

    # train, valid 개수랑 겹치는 부분

    

# fold0_train_path = "/opt/ml/detection/dataset/fold_test/fold0_train.json"
# fold0_val_path = "/opt/ml/detection/dataset/fold_test/fold0_val.json"

# with open(fold0_train_path, 'w') as f:
#     json.dump(dict())


In [8]:

distrs = [get_distribution(train_y)]
index = ['training set']

for fold_ind, (dev_ind, val_ind) in enumerate(stratified_group_k_fold(train_x, train_y, groups, k=5)):
    # dev_ind, val_ind는 list 형태로 들어가서 Series형식의 id, Image id를 뽑는다.
    dev_y, val_y = train_y[dev_ind], train_y[val_ind] # train index, 
    dev_groups, val_groups = groups[dev_ind], groups[val_ind] # Image id index,
    dev_id, val_id = train_x[dev_ind], train_x[val_ind]

    assert len(set(dev_groups) & set(val_groups)) == 0 # 가정 설정문,  동일한게 image file이 있는 지 확인 True이면 그대로 진행 아니라면 Assertion Error 생성
    
    distrs.append(get_distribution(dev_y))
    index.append(f'development set - fold {fold_ind}')
    distrs.append(get_distribution(val_y))
    index.append(f'validation set - fold {fold_ind}')

    #if fold_ind == 0:
    #print(dev_id)
    dev_id_json = list(set(dev_id))
    val_id_json = list(set(val_id))
    
    make_json(fold_ind, dev_id_json, val_id_json)
    

In [9]:

print('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(np.max(train_y) + 1)]) # label의 구성 퍼센티지들을 나타낸다.

Distribution per class:


,Label 0,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7,Label 8,Label 9
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
development set - fold 0,17.14%,27.45%,3.87%,4.04%,4.24%,12.72%,5.46%,22.38%,0.69%,2.02%
validation set - fold 0,17.13%,27.43%,3.88%,4.06%,4.25%,12.71%,5.46%,22.36%,0.69%,2.03%
development set - fold 1,17.14%,27.45%,3.87%,4.05%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
validation set - fold 1,17.12%,27.44%,3.89%,4.04%,4.25%,12.72%,5.46%,22.37%,0.69%,2.03%
development set - fold 2,17.14%,27.45%,3.88%,4.05%,4.25%,12.71%,5.46%,22.37%,0.69%,2.02%
validation set - fold 2,17.13%,27.44%,3.87%,4.04%,4.23%,12.72%,5.47%,22.38%,0.69%,2.03%
development set - fold 3,17.13%,27.44%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
validation set - fold 3,17.15%,27.46%,3.87%,4.04%,4.24%,12.71%,5.45%,22.38%,0.69%,2.01%
development set - fold 4,17.13%,27.44%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%


In [3]:
import pandas as pd

In [4]:
submission_dir = "./fold4_retinanet_resnet50_fpn_8_0.3.csv"

In [31]:
df = pd.read_csv(submission_dir)
print(df.head())
print(df["PredictionString"].head())

data_idx = 1
img_id = df["image_id"][data_idx]

predict_string = df["PredictionString"].iloc[data_idx].split(" ")[:-1]

for idx, coord in enumerate(predict_string):
    if idx % 5 == 0:



#print(predict_string)

SyntaxError: unexpected EOF while parsing (2873370343.py, line 17)